
# Amazon Bedrock Knowledge Bases로 구현하는 Structured RAG: Amazon Redshift DB를 사용하는 엔드 투 엔드 예제

Structured RAG는 Amazon Bedrock Knowledge Bases 고객이 자연어로 Redshift의 구조화된 데이터를 질의하고, 데이터 요약이 포함된 자연어 응답을 받아 질문에 대한 답을 얻을 수 있도록 해줍니다.

Amazon Bedrock Knowledge Bases는 고급 자연어 처리를 사용해 자연어 질의를 SQL 질의로 변환하므로, 사용자는 데이터를 이동하거나 사전 처리하지 않고도 소스에서 직접 데이터를 가져올 수 있습니다. 정확한 SQL 질의를 생성하기 위해 Bedrock Knowledge Base는 데이터 소스에 대해 제공된 데이터베이스 스키마, 이전 질의 이력, 기타 컨텍스트 정보를 활용합니다. 자세한 내용은 [문서](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-build-structured.html)를 참고하세요.

이 노트북은 Amazon Bedrock Knowledge Bases와 Redshift를 사용해 Structured RAG를 구축하는 샘플 코드를 제공합니다.

#### 절차:
- Amazon Redshift 데이터에 접근할 수 있는 필수 권한을 포함한 Knowledge Base 실행 역할을 생성합니다.
- Structured database(Redshift 데이터베이스)를 사용하는 knowledge base를 생성합니다.
- knowledge base 안에 데이터 소스를 생성합니다.
- KB API를 사용해 구조화된 데이터베이스의 메타데이터를 읽어 들이는 ingestion 작업을 시작합니다.

메타데이터 추출과 적재가 완료되면 사용자는 자연어 질의를 통해 Amazon Bedrock Knowledge Base API로 구조화된 데이터베이스와 상호작용할 수 있습니다.



#### 사전 준비 사항
이 노트북을 실행하려면 다음이 필요합니다:
- 워크그룹이 있는 Redshift Serverless 클러스터 또는 Redshift 프로비저닝 클러스터
- 워크그룹 또는 클러스터에 구조화된 데이터가 이미 적재되어 있어야 합니다.
- IAM Role, Secrets Manager 사용자 자격 증명, 또는 DB 사용자를 미리 구성해야 합니다.

사전 준비 사항에 대한 자세한 내용은 [문서](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-prereq-structured.html)를 참고하세요.



### 0 - 설정
이후 셀을 실행하기 전에 아래 셀을 먼저 실행하여 필요한 라이브러리를 설치하고 Bedrock에 연결하세요.

라이브러리를 설치하는 동안 pip 의존성 오류가 나타나더라도 무시해도 됩니다.


In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# %pip install --upgrade boto3
import boto3
print(boto3.__version__)

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import warnings
warnings.filterwarnings('ignore')


이 코드는 설정 과정의 일부이며 다음 작업을 수행합니다:
- 상위 디렉터리를 Python 시스템 경로에 추가합니다.
- 이후 실행에 필요한 커스텀 모듈인 `utils`의 `BedrockStructuredKnowledgeBase`를 가져옵니다.


In [ ]:
import sys
import logging
from pathlib import Path

current_path = Path().resolve()
current_path = current_path.parent

if str(current_path) not in sys.path:
    sys.path.append(str(current_path))

# Print sys.path to verify
print(sys.path)

from utils.structured_knowledge_base import BedrockStructuredKnowledgeBase

boto3 클라이언트를 설정하고 초기화합니다.

In [ ]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session(region_name='us-east-1')
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

Knowledge Base 이름과 기본 모델을 초기화하고 구성합니다. 이 기본 모델은 구조화된 데이터 저장소에서 가져온 레코드를 기반으로 자연어 응답을 생성하는 데 사용됩니다.

In [29]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-structured-kb-{suffix}"
knowledge_base_description = "Sample Structured KB"


foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

Amazon Bedrock Knowledge Bases는 서비스 역할을 사용해 Knowledge Base를 구조화된 데이터 저장소에 연결하고, 해당 저장소에서 데이터를 가져오며, 사용자 질의와 데이터 저장소 구조를 기반으로 SQL 질의를 생성합니다. Redshift Serverless를 사용하는지 Redshift 프로비저닝 클러스터를 사용하는지에 따라 여러 접근 패턴이 있습니다.


<div class="alert alert-block alert-warning">
<b>참고:</b> 아래는 Redshift Serverless와 Redshift Provisioned Cluster에 따라 달라지는 여러 접근 패턴입니다.
<br><br>
1.   Secrets Manager + Redshift Serverless WorkGroup
<br> 
2.   IAM Role + Redshift Serverless WorkGroup 
<br>
3.   IAM Role + Redshift Cluster
<br>
4.   Secrets Manager + Redshift Cluster 
<br>
5.   DB user + Redshift Cluster
<br>

이 노트북에서는 이러한 접근 패턴을 모두 살펴봅니다.
</div>



다음 접근 패턴에 따라 필요한 변수만 초기화하면 됩니다:

1. Secrets Manager + Redshift Serverless WorkGroup  
`workgroup_id`  
`secretArn`

2.   IAM Role + Redshift Serverless WorkGroup  
`workgroup_id`  
`redshiftDBName`

3.   IAM Role + Redshift Provisioned Cluster  
`provisioned_cluster_identifier`  
`provisioned_cluster_dbname`

4.   Secrets Manager + Redshift Provisioned Cluster  
`provisioned_cluster_identifier`  
`provisioned_cluster_dbname`  
`provisionedSecretArn`

5.   DB user + Redshift Provisioned Cluster  
`provisioned_cluster_identifier`  
`provisioned_cluster_dbname`  
`databaseUser`

Knowledge base 구성과 생성된 역할은 이러한 매개변수를 사용해 필요한 구성을 수행합니다. 실행 중 필요한 추가 단계는 이후 절차에서 다룹니다.



#### Redshift 접근 패턴 선택
구성에 맞는 옵션을 선택하세요. 


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Display the choices to the user
print("Choose your Redshift access pattern:")
print("1. Secrets Manager + Redshift Serverless WorkGroup")
print("2. IAM Role + Redshift Serverless WorkGroup")
print("3. IAM Role + Redshift Provisioned Cluster")
print("4. Secrets Manager + Redshift Provisioned Cluster")
print("5. DB User + Redshift Provisioned Cluster")

# Create a text input widget
choice_widget = widgets.Text(placeholder="Enter your choice (1 Only)")

# Display the widget
display(choice_widget)


In [31]:
# Change below variables as needed

#Redshift Serverless Cluster configuration details
workgroup_id = '<enter-redshift-serverless-workgroup-id>'
redshiftDBName = "<enter-redshift-serverless-database-name"
workgroupArn =  f"arn:aws:redshift-serverless:{region}:{account_id}:workgroup/{workgroup_id}"
secretArn = "<enter-secret-key-arn>"

#Redshift Provisioned Cluster configuration details
provisioned_cluster_identifier = "<enter-provisioned-cluster-identifier>"
provisioned_cluster_dbname = "<enter-provisioned-cluster-database-name>"
#Secrets manager ARN , this value is required to use "Secrets Manager + Redshift Provisioned Cluster" access pattern 
provisionedSecretArn = "<enter-secret-key-arn>"
#Redshift Database UserName, this value is required to use "DB User + Redshift Provisioned Cluster" access pattern
databaseUser = "<enter-db-username>"

# kb Configuration
kbConfigParam = {
            "type": "SQL",
            "sqlKnowledgeBaseConfiguration": {
                "type": "REDSHIFT",
                "redshiftConfiguration": {
                    "storageConfigurations": [{
                        "type": "REDSHIFT",
                        "redshiftConfiguration": {
                            "databaseName": redshiftDBName
                        }
                    }],
                    "queryEngineConfiguration": {
                        "type": "SERVERLESS",
                        "serverlessConfiguration": {
                            "workgroupArn": workgroupArn,
                            "authConfiguration": {}
                        }
                    }
                }
            }
        }

kbProvisionedConfigParam = {
    "type": "SQL",
    "sqlKnowledgeBaseConfiguration": {
        "type": "REDSHIFT",
            "redshiftConfiguration": {
                "queryEngineConfiguration": {
                    "type": "PROVISIONED",
                    "provisionedConfiguration": {
                        "authConfiguration": {},
                        "clusterIdentifier": provisioned_cluster_identifier
                    }
                },
                "storageConfigurations": [{
                        "redshiftConfiguration": {
                            "databaseName": provisioned_cluster_dbname
                        },
                        "type": "REDSHIFT"
                }]
            } 
        }  
}


### 1 - Knowledge Base 생성

선택한 접근 패턴에 따라 
1. 이 코드는 Knowledge Base 구성을 추가 매개변수로 업데이트합니다. 예를 들어 `IAM Role + Redshift Serverless` 접근 패턴을 선택하면 `[authCofiguration][type]` 매개변수가 "IAM"으로 업데이트됩니다.
2. 그런 다음 Knowledge Base를 생성합니다 


In [ ]:
# Access the entered value
access_pattern_choice = int(choice_widget.value)
try:
    access_pattern_choice = int(choice_widget.value)
    if access_pattern_choice ==1:
        print(f"Access pattern:{choice_widget.value}. Secrets Manager + Redshift Serverless WorkGroup")

        kbConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['serverlessConfiguration']['authConfiguration']['type'] = "USERNAME_PASSWORD"
        kbConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['serverlessConfiguration']['authConfiguration']['usernamePasswordSecretArn'] = secretArn

        knowledge_base = BedrockStructuredKnowledgeBase(
                        kb_name=f'{knowledge_base_name}',
                        kb_description=knowledge_base_description,
                        workgroup_arn=workgroupArn,
                        secrets_arn = secretArn,
                        kbConfigParam = kbConfigParam,
                        suffix = f'{suffix}-f'
                    )
       
    elif access_pattern_choice ==2:
        print(f"Access pattern:{choice_widget.value} IAM Role + Redshift Serverless WorkGroup")

        kbConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['serverlessConfiguration']['authConfiguration']['type'] = "IAM"
        
        knowledge_base = BedrockStructuredKnowledgeBase(
                        kb_name=f'{knowledge_base_name}',
                        kb_description=knowledge_base_description,
                        workgroup_arn=workgroupArn,
                        kbConfigParam = kbConfigParam,
                        suffix = f'{suffix}-f'
                    )
    
    elif access_pattern_choice == 3:
        print(f"Access pattern:{choice_widget.value} IAM Role + Redshift Provisioned Cluster")

        kbProvisionedConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['provisionedConfiguration']['authConfiguration']['type'] = "IAM"
        
        knowledge_base = BedrockStructuredKnowledgeBase(
                        kb_name=f'{knowledge_base_name}',
                        kb_description=knowledge_base_description,
                        cluster_identifier=provisioned_cluster_identifier,
                        db_name=provisioned_cluster_dbname,
                        kbConfigParam = kbProvisionedConfigParam,
                        suffix = f'{suffix}-f'
                    )
        
    elif access_pattern_choice == 4:
        print(f"Access pattern:{choice_widget.value} Secrets Manager + Redshift Provisioned Cluster")

        kbProvisionedConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['provisionedConfiguration']['authConfiguration']['type'] = "USERNAME_PASSWORD"
        kbProvisionedConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['provisionedConfiguration']['authConfiguration']['usernamePasswordSecretArn'] = provisionedSecretArn
        
        knowledge_base = BedrockStructuredKnowledgeBase(
                        kb_name=f'{knowledge_base_name}',
                        kb_description=knowledge_base_description,
                        cluster_identifier=provisioned_cluster_identifier,
                        db_name=provisioned_cluster_dbname,
                        secrets_arn = provisionedSecretArn,
                        kbConfigParam = kbProvisionedConfigParam,
                        suffix = f'{suffix}-f'
                    )  
    elif access_pattern_choice == 5:
        print(f"Access pattern:{choice_widget.value} DB User + Redshift Provisioned Cluster")

        kbProvisionedConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['provisionedConfiguration']['authConfiguration']['type'] = "USERNAME"
        kbProvisionedConfigParam['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['provisionedConfiguration']['authConfiguration']['databaseUser'] = databaseUser

        knowledge_base = BedrockStructuredKnowledgeBase(
                        kb_name=f'{knowledge_base_name}',
                        kb_description=knowledge_base_description,
                        cluster_identifier=provisioned_cluster_identifier,
                        db_name=provisioned_cluster_dbname,
                        db_user=databaseUser,
                        kbConfigParam = kbProvisionedConfigParam,
                        suffix = f'{suffix}-f'
                    )   
    else:
        print("Invalid choice. Please enter a number between 1 and 2.")
except ValueError:
    print("Invalid input. Please enter a number.")


### IAM 접근 패턴의 경우, 인증에 사용하는 역할에 데이터베이스 접근 권한 부여

아래 예제 SQL 문을 사용해 사용자를 생성하고 권한을 부여할 수 있습니다. 데이터 집합에 필요한 접근 수준에 맞게 GRANT 문을 수정하세요. 

자세한 단계는 [문서](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-prereq-structured.html#knowledge-base-prereq-structured-db-access)를 참고하세요.


In [ ]:
#Following SQL commands should be executed in Redshift Query Editor
print(f'CREATE USER "IAMR:{knowledge_base.bedrock_kb_execution_role_name}" WITH PASSWORD DISABLE;')
print(f'GRANT SELECT ON ALL tables IN SCHEMA public TO "IAMR:{knowledge_base.bedrock_kb_execution_role_name}";')


### 2 - Ingestion 작업 시작

이 단계에서는 데이터 소스를 동기화하기 위해 ingestion 작업을 시작합니다. 


In [ ]:
# ensure that the kb is available
time.sleep(60)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id


### 3 - Structured Knowledge Base 테스트
이제 Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html), [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html), 그리고 [**generate_query**](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_GenerateQuery.html) 함수를 사용해 시험해 볼 수 있습니다. 

[**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)를 사용하면 응답에 SQL 질의 실행 결과가 포함됩니다. 

[**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)를 사용하면 SQL 질의 실행 결과를 기반으로 생성된 응답을 받습니다.

[**generate_query**](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_GenerateQuery.html) API를 사용하면 자연어 질의를 SQL로 변환합니다.



In [ ]:
query = "<Enter your sample query here>"

#### 3.1 - RetrieveAndGenerate API 사용

In [ ]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

#### 3.2 - Retrieve API 사용

In [40]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": query
    }
)


In [ ]:
import json
import pandas as pd

# Function to extract retrieved results from Retrieve API response into a pandas dataframe.

def response_print(retrieve_resp):

    # Extract the retrievalResults list
    retrieval_results = retrieve_resp['retrievalResults']

    # Dictionary to store the extracted data
    extracted_data = {}

    # Iterate through each item in retrievalResults
    for item in retrieval_results:
        row = item['content']['row']
        for col in row:
            column_name = col['columnName']
            column_value = col['columnValue']
            
            # If this column hasn't been seen before, create a new list for it
            if column_name not in extracted_data:
                extracted_data[column_name] = []
            
            # Append the value to the appropriate list
            extracted_data[column_name].append(column_value)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(extracted_data)

    return df
    

In [ ]:
# Display the Retrieved results records
df = response_print(response_ret)
print(df.head())

#### 3.3 - Generate Query 사용

In [ ]:
query_response = bedrock_agent_runtime_client.generate_query(
    queryGenerationInput={
        "text": query,
        "type": "TEXT"
    },
    transformationConfiguration={
        "mode" : "TEXT_TO_SQL",
        "textToSqlConfiguration": {
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseArn": knowledge_base.knowledge_base['knowledgeBaseArn']
            }
        }
    }
)

generated_sql = query_response['queries'][0]['sql']
generated_sql


### 정리
아래 섹션의 주석을 해제한 뒤 실행해 리소스를 모두 삭제하세요


In [ ]:
# # Delete resources
# print("===============================Deleteing resources ==============================\n")
knowledge_base.delete_kb( delete_iam_roles_and_policies=True)